# Weekly

In [1]:
from strategy.weekly_price import WeeklyPrice
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from backtester.backtester import Backtester as b
from database.market import Market
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from modeler.modeler import Modeler as m
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import math
import pickle

In [2]:
## Loading Constants
start = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
strat = WeeklyPrice()
qfc = QuarterlyFinancialCategorical()

In [3]:
categories = qfc.retrieve_data("portfolio_stock_category_sim")
models = strat.retrieve_data("portfolio_weekly_models")

In [4]:
models

,_id,api,model,score,model_type,category,year,quarter
0,60b5b0432239a3346565c613,xgb,b'\x80\x04\x95d\x02\x00\x00\x00\x00\x00\x00\x8...,0.993649,regression,100Industrials,2021,1
1,60b5b0432239a3346565c614,xgb,b'\x80\x04\x95\xab\x03\x00\x00\x00\x00\x00\x00...,0.705714,classification,100Industrials,2021,1
2,60b5b04a2239a3346565c616,xgb,b'\x80\x04\x95b\x02\x00\x00\x00\x00\x00\x00\x8...,0.992173,regression,250Consumer Discretionary,2021,1
3,60b5b04a2239a3346565c617,xgb,b'\x80\x04\x95\xab\x03\x00\x00\x00\x00\x00\x00...,0.711538,classification,250Consumer Discretionary,2021,1
4,60b5b0532239a3346565c619,xgb,b'\x80\x04\x95b\x02\x00\x00\x00\x00\x00\x00\x8...,0.992560,regression,250Information Technology,2021,1
...,...,...,...,...,...,...,...,...
125,60e2d03b4fc67fd01dd264ca,xgb,b'\x80\x04\x95\xab\x03\x00\x00\x00\x00\x00\x00...,0.567568,classification,250Communication Services,2021,3
126,60e2d03d4fc67fd01dd264cc,xgb,b'\x80\x04\x95b\x02\x00\x00\x00\x00\x00\x00\x8...,0.939811,regression,250Energy,2021,3
127,60e2d03d4fc67fd01dd264cd,xgb,b'\x80\x04\x95\xfe\t\x00\x00\x00\x00\x00\x00\x...,0.692308,classification,250Energy,2021,3
128,60e2d03f4fc67fd01dd264cf,xgb,b'\x80\x04\x95d\x02\x00\x00\x00\x00\x00\x00\x8...,0.698537,regression,250Utilities,2021,3


In [5]:
categories.columns

Index(['_id', 'year', 'quarter', 'ticker', 'prediction'], dtype='object')

In [6]:
datetime.now()

datetime.datetime(2021, 8, 2, 11, 33, 32, 326075)

In [7]:
market.connect()
classification = market.retrieve_data("portfolio_dataset_week_classification")
regression = market.retrieve_data("portfolio_dataset_week_regression")
market.close()

In [8]:
gap = 5
week_gap = int(gap/5)
training_years = 1
quarterly_range = range(3,4)
yearly_range = range(2021,2022)

In [9]:
products = []
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        quarter_categories = categories[(categories["year"] == year) & (categories["quarter"] == quarter)]
        quarter_models = models[(models["year"]==year) & (models["quarter"]==quarter)]
        for category in tqdm(quarter_categories["prediction"].unique()):
            try:
                group_tickers = quarter_categories[(quarter_categories["prediction"] == category)]
                group_models = quarter_models[(quarter_models["category"] == category)]
                prices = []
                if group_tickers.index.size > 0:
                    for ticker in group_tickers["ticker"].unique():
                        try:
                            ticker_class = classification.copy()[["year","quarter","week",ticker]].rename(columns={ticker:"y_class"})
                            ticker_regr = regression.copy()[["year","quarter","week",ticker]].rename(columns={ticker:str(0)})
                            test = ticker_regr
                            for i in range(1,14):
                                test[str(i)] = test[str(0)].shift(1)
                            f = test.dropna()
                            for col in ["year","quarter","week"]:
                                f[col] = [int(x) for x in f[col]]
                            f["date"] = [datetime(int(row[1]["year"]),int((row[1]["quarter"] * 3) - 2),1) for row in f.iterrows()]
                            f["ticker"] = ticker
                            prices.append(f)
                        except Exception as e:
                            print(str(e))
                            continue
                if len(prices) > 0:
                    p = pd.concat(prices)
                    prediction_data = p[p["date"]==datetime(year,int((quarter*3)-2),1)]
                    X = prediction_data.drop(["quarter","year","y_class","y","date","ticker"],axis=1,errors="ignore")
                    refined_data = {"X":X}
                    factors = refined_data["X"].columns
                    product = prediction_data[["year","quarter","week","ticker"]]
                    for i in range(group_models.index.size):
                        model = pickle.loads(group_models.iloc[i]["model"])
                        model_type = group_models.iloc[i]["model_type"]
                        score = group_models.iloc[i]["score"]
                        predictions = model.predict(prediction_data[factors])
                        product[f"weekly_price_categorical_{model_type}_prediction"] = predictions
                        product[f"weekly_price_categorical_{model_type}_score"] = score
                    products.append(product)
            except Exception as e:
                print(str(e))
                continue

  0%|                                                                       | 0/1 [00:00<?, ?it/s]

  0%|                                                                      | 0/23 [00:00<?, ?it/s]

  9%|█████▍                                                        | 2/23 [00:00<00:02,  8.51it/s]

 13%|████████                                                      | 3/23 [00:00<00:03,  5.08it/s]

 17%|██████████▊                                                   | 4/23 [00:01<00:05,  3.33it/s]

 22%|█████████████▍                                                | 5/23 [00:01<00:06,  2.94it/s]

 26%|████████████████▏                                             | 6/23 [00:01<00:04,  3.57it/s]

 30%|██████████████████▊                                           | 7/23 [00:01<00:04,  3.76it/s]

 35%|█████████████████████▌                                        | 8/23 [00:02<00:04,  3.58it/s]

 39%|████████████████████████▎                                     | 9/23 [00:02<00:05,  2.79it/s]



In [10]:
final = pd.concat(products)

In [11]:
final_weekly = final[(final["year"] == 2021) & (final["week"] > 29)]

In [12]:
final_weekly

,year,quarter,week,ticker,weekly_price_categorical_regression_prediction,weekly_price_categorical_regression_score,weekly_price_categorical_classification_prediction,weekly_price_categorical_classification_score
122,2021,3,30,AAL,22.545681,0.993949,1,0.600000
122,2021,3,30,AOS,71.089462,0.993949,1,0.600000
122,2021,3,30,CSX,33.525299,0.993949,1,0.600000
122,2021,3,30,EMR,99.556946,0.993949,1,0.600000
122,2021,3,30,EXPD,120.205338,0.993949,1,0.600000
...,...,...,...,...,...,...,...,...
122,2021,3,30,EA,141.810349,0.992990,0,0.567568
122,2021,3,30,FB,309.085785,0.992990,1,0.567568
122,2021,3,30,TMUS,141.694183,0.992990,1,0.567568
122,2021,3,30,PXD,138.752441,0.939811,0,0.692308


In [13]:
strat.db.connect()
strat.db.store_data("portfolio_weekly_predictions",final_weekly)
strat.db.close()

In [15]:
datetime.now().isocalendar()[1]

27